# Neural functional theory for inhomogeneous fluids - Tutorial

This material provides a pedagogical introduction to the concepts and methods presented in

**Neural functional theory of inhomogeneous fluids: Fundamentals and applications**  
Florian Sammüller, Sophie Hermann, Daniel de las Heras, and Matthias Schmidt (2023); [arxiv:2307.04539](https://arxiv.org/abs/2307.04539).

We show how the physics of simple fluids can be described via **many-body simulations** and **classical density functional theory** (DFT) and how the two approaches can be combined with the help of machine learning techniques to a **neural functional theory**.
Compared to the above publication, we take a step back and consider here a one-dimensional system of hard rods.
We demonstrate for this simple model fluid in the following the central methods of the different approaches and provide hands-on code examples and exercises.
Besides the theoretical overview given here, a more in-depth account of the physical and methodological background can be found in the accompanying manuscript:

**Why neural functionals suit statistical mechanics**  
Florian Sammüller, Sophie Hermann, and Matthias Schmidt (2023).

## Part 1: Many-body simulations

Many-body simulations have long become a standard tool for the investigation of classical fluids.
Conceptually, they offer a rather straightforward way to predict the behavior of a fluid from a microscopic description, i.e. by specifying the interactions of its constituent particles.
This comes often comes at the significant expense of computational resources.

In the following, we focus on Monte Carlo methods for the description of equilibrium systems.
Specifically, we choose the grand ensemble where the temperature $T$ and the chemical potential $\mu$ are kept fixed (we also specify the length $L$ of our one-dimensional domain).
The statistical mechanics of such a system is then given as follows: